In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

data1 = pd.read_csv('./01.csv',)
data1 = data1.loc[:,data1.columns[2:]]

i = 2
while i <= 5:
    data = pd.read_csv('./0%d.csv'%i)
    data = data.loc[:,data.columns[2:]]
    data1 = pd.concat([data1,data])
    i = i +1

del data

ids = data1.business_id.unique()
np.savetxt('id.txt',ids,delimiter='\n')

stars = []
reviews = []
for i in tqdm(range(len(ids))):
    j = ids[i]
    tmp = data1[data1.business_id == j]
    star = [x for x in tmp.stars.values]
    review = ''
    for y in tmp.text.values:
        review = review + str(y) + ' '
    stars.append(star)
    reviews.append(review)

final = pd.DataFrame({'business_id':ids,'stars':stars,'reviews':reviews})
del data1

del ids,stars,reviews

final.to_csv('final.csv')
final = pd.read_csv('final.csv')

from gensim.models.phrases import Phrases, Phraser

phrases = Phrases(final.reviews.values, min_count=10, threshold=10)
bigram = Phraser(phrases)

bireviews = bigram[final.reviews.values]
final.reviews = bireviews

final.to_csv('bifinal.csv')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
tfidf = TfidfVectorizer(stop_words = 'english')
bifinal = pd.read_csv('bifinal.csv',index_col=0)
reviews = bifinal.reviews
X = tfidf.fit_transform(reviews)
names = tfidf.get_feature_names()
with open('features.txt','w') as f:
    for x in names:
        f.write(x + '\n')
def save_sparse_csr(filename, array):
    
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)
save_sparse_csr(filename='tfidf',array=X)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
tfidf = TfidfVectorizer(stop_words = 'english')
bifinal = pd.read_csv('bifinal.csv',index_col=0)
reviews = bifinal.reviews
X = tfidf.fit_transform(reviews)
names = tfidf.get_feature_names()
with open('features.txt','w') as f:
    for x in names:
        f.write(x + '\n')

def save_sparse_csr(filename, array):
    
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)
save_sparse_csr(filename='tfidf',array=X)
import numpy as np
from scipy.sparse import csr_matrix
import re
with open('features.txt') as f:
    name = f.readlines()
names = [re.sub('\n','',x) for x in name]

def load_sparse_csr(filename):
    
    loader = np.load(filename + '.npz')
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])
X = load_sparse_csr('tfidf')



features = []
for i in range(5000):
    feature = []
    tmp = X.getrow(i).toarray()[0]
    for x in range(10):
        large = tmp.argmax()
        tmp[large] = 0
        feature.append(names[large])
    features.append(feature)
with open('1.txt','w') as f:
    for line in features:
        for i in range(10):
            if i != 9:
                f.write(line[i] + ',')
            else:
                f.write(line[i] + '\n')

features = []
for i in range(X.shape[0]):
    feature = []
    tmp = X.getrow(i).toarray()
    tuples = sorted(enumerate(tmp), key=lambda x: x[1])[:5]
    for t in tuples:
        feature.append(names[t[0]])
    features.append(feature)

with open('features.txt','w') as f:
    for x in names:
        f.write(x + '\n')

import numpy as np
from scipy.sparse import csr_matrix

def save_sparse_csr(filename, array):
    
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)
    
def load_sparse_csr(filename):
    
    loader = np.load(filename + '.npz')
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

save_sparse_csr(filename='tfidf',array=X)

X = load_sparse_csr('tfidf')

features = []
for i in range(X.shape[0]):
    feature = []
    tmp = X.getrow(i).toarray()[0]
    tuples = sorted(enumerate(tmp), reverse=True,key=lambda x: x[1])[:5]
    for t in tuples:
        feature.append(names[t[0]])
    features.append(feature)
bifinal = bifinal.drop('reviews',axis=1)
bifinal['top'] = features
bifinal.to_csv('final.csv')

features = []
for i in tqdm(range(X.shape[0])):
    feature = []
    tmp = X.getrow(i).toarray()[0]
    tuples = sorted(enumerate(tmp), reverse=True,key=lambda x: x[1])[:5]
    for t in tuples:
        feature.append(names[t[0]])
    features.append(feature)

with open('features.txt') as f:
    name = f.readlines()

names = [re.sub('\n','',x) for x in name]

restaurants = pd.read_csv('../../yaoshen.csv',index_col=0)

restaurants = restaurants[restaurants.business_id.isin(ids)]

fastfood = restaurants[restaurants.fastfood==1].business_id
bars = restaurants[restaurants.bars==1].business_id
japan = restaurants[restaurants.japan==1].business_id
asian = restaurants[restaurants.asian==1].business_id

feature1 = ['wheelchair','street','cater','waiter','service','waitress','lunch','lot','parking','night','late','reservation']
feature2 = ['hipster','late','night','cater','service','waiter','waitress','attire','dressing','kid','dinner','dog']
feature3 = ['alcohol','beer','wine','group','party','reservation','street','noise']
feature4 = ['noise','intimate','friend','music']


reviews = pd.read_csv('bifinal.csv')

reviews = reviews[['business_id','stars','reviews']]

import re
from nltk.tokenize import word_tokenize

WC = []
for i in range(4):
    if i == 0:
        index = fastfood
        feature = feature1
    elif i==1:
        index = bars
        feature = feature2
    elif i == 2:
        index = japan
        feature = feature3
    elif i==3:
        index = asian
        feature = feature4
    tmp = reviews[reviews.business_id.isin(index)]
    wc = []
    for f in feature:
        words = []
        for review in tmp.reviews:
            line = {}
            finds = re.findall('[a-z]* [a-z]* '+f+' [a-z]* [a-z]*',review)
            for find in finds:
                find = re.sub(f,'',find)
                tokens = word_tokenize(find)
                for token in tokens:
                    if token not in line.keys():
                        line[token] = 1
                    else:
                        line[token] = line[token] + 1
            words.append(line)
        wc.append(words)
    WC.append(wc)


restaurants['features'] = ''

from tqdm import tqdm

restaurants= restaurants.reset_index(drop=True)

restaurants = restaurants.set_index(restaurants.business_id).loc[ids]

restaurants = restaurants.reset_index(drop=True)

a= 0
b=0
c=0
d=0
for x in tqdm(range(restaurants.shape[0])):
    
    if restaurants.loc[x,'fastfood'] == 1:
        restaurants.loc[x,'features'] += 'FASTFOOD: '
        for j in range(len(WC[0])):
            f = ''
            f = '; '.join([y[0]+': '+str(y[1]) for y in WC[0][j][a].items() if len(WC[0][j][a].items()) > 0])
            restaurants.loc[x,'features'] += feature1[j].upper() + ': ' + f + ' '
        a += 1
    if restaurants.loc[x,'bars'] == 1:
        restaurants.loc[x,'features'] += 'BARS: '
        for j in range(len(WC[1])):
            f = ''
            f = '; '.join([y[0]+': '+str(y[1]) for y in WC[1][j][b].items() if len(WC[1][j][b].items()) > 0])
            restaurants.loc[x,'features'] += feature2[j].upper() + ': ' + f + ' '
        b +=1
    if restaurants.loc[x,'japan'] == 1:
        restaurants.loc[x,'features'] += 'JAPAN: '
        for j in range(len(WC[2])):
            f = ''
            f = '; '.join([y[0]+': '+str(y[1]) for y in WC[2][j][c].items() if len(WC[2][j][c].items()) > 0])
            restaurants.loc[x,'features'] += feature3[j].upper() + ': ' + f + ' '
        c +=1
    if restaurants.loc[x,'asian'] == 1:
        restaurants.loc[x,'features'] += 'ASIAN: '
        for j in range(len(WC[3])):
            f = ''
            f = '; '.join([y[0]+': '+str(y[1]) for y in WC[3][j][d].items() if len(WC[3][j][d].items()) > 0])
            restaurants.loc[x,'features'] += feature4[j].upper() + ': ' + f + ' '
        d +=1

final = pd.read_csv('finaldata.csv')

final.set_index('business_id').join(restaurants.set_index('business_id')[['fastfood','bars','japan','asian','features']]).to_csv('finaldata2.csv')


final2 = pd.read_csv('finaldata2.csv')

features = final2.features.values

In [ ]:
from nltk import pos_tag

In [ ]:
new = []
for feature in tqdm(features):
    tokens = word_tokenize(str(feature))
    out = ''
    yes = 0
    for token in tokens:
        if token.isupper():
            out = out + token +': '
        elif pos_tag([token])[0][1].startswith('J'):
            out = out + token + ': '
            yes = 1
        elif yes == 1 and token != ':':
            out = out + token + '; '
            yes = 0
    new_out=''.join([x for x in re.findall('[A-Z:]+[a-z: 0-9;]+',out) if x.upper()!= x])
    new.append(new_out)

In [ ]:
final2.features = new

In [ ]:
comments = pd.read_csv('comments.csv')

In [ ]:
final3 = final2.set_index('business_id').join(comments.set_index('ID'),how = 'right')

In [ ]:
def transform(feature):
    categories = re.findall('[A-Z]+',str(feature))
    features = []
    for x in categories:
        find = re.findall(x + '[: ;0-9a-z]+',feature)
        find = re.sub('([A-Z]+)[:]','\\1',find[0])
        features.append(find)
    return '\n'.join(features)
final3.features = final3.features.apply(transform)

In [ ]:
final3.features.values[1]

In [ ]:
final3.to_csv('final3.csv')